In [3]:
import xgboost as xgb
import boto3
boto3.setup_default_session(profile_name='ml-staging-admin')
# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client(
    "sagemaker-runtime", region_name='us-east-1',)

# The endpoint name must be unique within 
# an AWS Region in your AWS account. 




In [1]:
from inference import load_model
model = load_model("model_dir/model.pth")
model

CollaborativeFilteringModel(
  (user_embedding): Embedding(50000, 4)
  (post_embedding): Embedding(1800, 4)
)

In [5]:
endpoint_name='pytorch2'
payload = """
{
    "user_ids": ["f34d7147-9d39-4145-bc7e-3a3c05029035", "46b5b420-3317-4a14-9c90-9dc4206cf960"],
    "media_ids": ["01HEKK737WXFC9TWQ036XDS0QH", "01H2K82V8XCKGD882FPQ3TQ9WF"]
}
"""

response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=payload,
        ContentType="application/json",
    )

In [6]:
response

{'ResponseMetadata': {'RequestId': '8446eaad-d4da-4ea7-ac30-0b954a23491c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8446eaad-d4da-4ea7-ac30-0b954a23491c',
   'x-amzn-invoked-production-variant': 'variant-name-1',
   'date': 'Mon, 04 Mar 2024 19:14:04 GMT',
   'content-type': 'application/json',
   'content-length': '41',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'variant-name-1',
 'Body': <botocore.response.StreamingBody at 0x7f13597e1750>}

In [7]:
response_body_str = response['Body'].read().decode('utf-8')
response_body_str

'[0.14748752117156982, 0.8654617071151733]'